## Requirements
It is expected that simulator can perform following:

- initialize state
- read program, and for each gate:
    - calculate matrix operator
    - apply operator (modify state)

- perform multi-shot measurement of all qubits using weighted random technique

It is up to you how you will organize code, but this is our suggestion:

### Input format (quantum program)

It is enough if simulator takes program in following format:

```
[
  { \"unitary\": [[0.70710678, 0.70710678], [0.70710678, -0.70710678]], \"target\": [0] },
  { \"unitary\": [ [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0] ], \"target\": [0, 1] }
  ...
]
```

Or, you can define unitaries in the code, and accept program with gate names instead:

```
[
  { \"gate\": \"h\", \"target\": [0] },
  { \"gate\": \"cx\", \"target\": [0, 1] }
  ...
]
```

In [1]:
import numpy as np
import unittest
from numpy.random import multinomial
import math
import json
import functools

# Helper functions

In [2]:
def ln2(number):
    return math.frexp(number)[1] - 1

In [3]:
class TestHelpers(unittest.TestCase):

    def test_ln2_2(self):
        self.assertEqual(ln2(2), 1)

    def test_ln2_8(self):
        self.assertEqual(ln2(8), 3)
    def test_ln_2(self):
        self.assertEqual(ln2(256), 8)

suite = unittest.TestLoader().loadTestsFromTestCase(TestHelpers)
unittest.TextTestRunner().run(suite)

...
----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

# Get ground state [DONE]

In [4]:
def get_ground_state(num_qubits):
    # return vector of size 2**num_qubits with all zeroes except first element which is 1

#     assert num_qubits > 0
    
    ground_state = a = np.zeros(2**num_qubits)
    ground_state[0] = 1
    return ground_state

In [5]:
class TestGetGroundState(unittest.TestCase):

#     def test_invalid_input(self):
#         self.assertRaises(get_ground_state(0), AssertionError)

    def test_single_qubit(self):
        self.assertSequenceEqual(
            list(get_ground_state(1)),
            list(np.array([1, 0]))
        )

    def test_multi_qubit(self):
        self.assertSequenceEqual(
            list(get_ground_state(3)),
            list(np.array([1, 0, 0, 0, 0 ,0, 0, 0]))
        )

suite = unittest.TestLoader().loadTestsFromTestCase(TestGetGroundState)
unittest.TextTestRunner().run(suite)

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

# Get Operator [not started]

In [6]:
# Define 2x2 Identity
I = np.identity(2)

# Define X gate
X = np.array([
[0, 1],
[1, 0]
])

# Define Y gate
Y = np.array([
[0, 0-1j],
[0+1j, 0]
])

# Define Z gate
Z = np.array([
[1, 0],
[0, -1]
])

# Define H gate
H = np.array([
[1, 1],
[1, -1]
])/math.sqrt(2)

# Define projection operator |0><0|
P0x0 = np.array([
[1, 0],
[0, 0]
])

# Define projection operator |1><1|
P1x1 = np.array([
[0, 0],
[0, 1]
])

KNOWN_GATES = {
    "I": I,
    "X": X,
    "Y": Y,
    "Z": Z,
    "H": H,
    "P0": P0x0,
    "P1": P1x1
}

In [7]:
def get_operator(total_qubits, gate_unitary, target_qubits):
    # return unitary operator of size 2**n x 2**n for given gate and target qubits
    
    def get_gate_from_name(name):
        upper_case_name = name.upper()
        if upper_case_name[0] == 'C':
            return KNOWN_GATES[upper_case_name[1:]]
        return KNOWN_GATES[upper_case_name]
    
    def multiply_gates(gate_unitary, target_qubits, projector=None, control_qubit=None):
        gates = [I] * total_qubits
        for target_qubit in target_qubits:
            gates[target_qubit] = gate_unitary
        if projector is not None and control_qubit is not None:
            gates[control_qubit] = projector
        return functools.reduce(lambda a,b : np.kron(a, b), gates)
    
    is_a_controlled_gate = gate_unitary.upper()[0] == 'C'
    gate = get_gate_from_name(gate_unitary)
    
    if is_a_controlled_gate:
        result\
        = multiply_gates(I, [target_qubits[1]], P0x0, target_qubits[0])\
        + multiply_gates(gate, [target_qubits[1]], P1x1, target_qubits[0])
#         print('controlled-gate:\n', np.array2string(result))
        return result
    else:
        result = multiply_gates(gate, target_qubits)
#         print('gate:\n', np.array2string(result))
        return result
    
    return gate_unitary

In [8]:
class TestGetOperator(unittest.TestCase):

    def test_n_is_1_X_on_0(self):
        np.testing.assert_array_equal(
            get_operator(1, "X", [0]),
            np.array([
                [0,1],
                [1,0]
            ])
        )

    def test_n_is_2_X_on_0(self):
        np.testing.assert_array_equal(
            get_operator(2, "X", [0]),
            np.array([
                [0,0,1,0],
                [0,0,0,1],
                [1,0,0,0],
                [0,1,0,0]
            ])
        )
    
    def test_n_is_2_X_on_1(self):
        np.testing.assert_array_equal(
            get_operator(2, "X", [1]),
            np.array([
                [0,1,0,0],
                [1,0,0,0],
                [0,0,0,1],
                [0,0,1,0]
            ])
        )

    def test_n_is_2_H_on_0_and_1(self):
        np.testing.assert_array_equal(
            get_operator(2, "H", [0,1]),
            np.array([
                [1,1,1,1],
                [1,-1,1,-1],
                [1,1,-1,-1],
                [1,-1,-1,1]
            ])/2
        )
        
    def test_n_is_3_X_on_2(self):
        np.testing.assert_array_equal(
            get_operator(3, "X", [2]),
            np.array([
                [0, 1, 0, 0, 0, 0, 0, 0,],
                [1, 0, 0, 0, 0, 0, 0, 0,],
                [0, 0, 0, 1, 0, 0, 0, 0,],
                [0, 0, 1, 0, 0, 0, 0, 0,],
                [0, 0, 0, 0, 0, 1, 0, 0,],
                [0, 0, 0, 0, 1, 0, 0, 0,],
                [0, 0, 0, 0, 0, 0, 0, 1,],
                [0, 0, 0, 0, 0, 0, 1, 0,]
            ])
        )

    def test_n_is_3_CNOT_on_0_and_2(self):
        np.testing.assert_array_equal(
            get_operator(3, "CX", [0,2]),
            np.array([
                [1, 0, 0, 0, 0, 0, 0, 0],
                [0, 1, 0, 0, 0, 0, 0, 0],
                [0, 0, 1, 0, 0, 0, 0, 0],
                [0, 0, 0, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, 1, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 1],
                [0, 0, 0, 0, 0, 0, 1, 0]
            ])
        )
        
suite = unittest.TestLoader().loadTestsFromTestCase(TestGetOperator)
unittest.TextTestRunner().run(suite)

.F....
FAIL: test_n_is_2_H_on_0_and_1 (__main__.TestGetOperator)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-8-182ff02c0409>", line 35, in test_n_is_2_H_on_0_and_1
    np.testing.assert_array_equal(
  File "/opt/conda/lib/python3.8/site-packages/numpy/testing/_private/utils.py", line 930, in assert_array_equal
    assert_array_compare(operator.__eq__, x, y, err_msg=err_msg,
  File "/opt/conda/lib/python3.8/site-packages/numpy/testing/_private/utils.py", line 840, in assert_array_compare
    raise AssertionError(msg)
AssertionError: 
Arrays are not equal

Mismatched elements: 16 / 16 (100%)
Max absolute difference: 1.11022302e-16
Max relative difference: 2.22044605e-16
 x: array([[ 0.5,  0.5,  0.5,  0.5],
       [ 0.5, -0.5,  0.5, -0.5],
       [ 0.5,  0.5, -0.5, -0.5],
       [ 0.5, -0.5, -0.5,  0.5]])
 y: array([[ 0.5,  0.5,  0.5,  0.5],
       [ 0.5, -0.5,  0.5, -0.5],
       [ 0.5,  0.5, -0.5, -0.5]

<unittest.runner.TextTestResult run=6 errors=0 failures=1>

# Run Program [not started]


In order to implement simulator, it is best if you have function which returns operator for any unitary targeting any qubit(s) for any circuit size, something like:

```get_operator(total_qubits, gate_unitary, target_qubits)```

But this is not trivial so it is enough if you can implement it for any 1-qubit gates and CNOT only.

If you are still enthusiastic and you wish to implement universal operator function then please refer to:

- [qosf-simulator-task-additional-info.pdf|https://github.com/quantastica/qosf-mentorship/blob/master/qosf-simulator-task-additional-info.pdf]

- Book Nielsen, Michael A.; Chuang, Isaac (2000). Quantum Computation and Quantum Information, 10th Anniversary Edition, Section 8.2.3, Operator-Sum Representation




In [21]:
def run_program(initial_state, program):
    # read program, and for each gate:
    #   - calculate matrix operator
    #   - multiply state with operator
    # return final state

    def get_gate_from_name(name):
        upper_case_name = name.upper()
        if upper_case_name[0] == 'C':
            return KNOWN_GATES[upper_case_name[1:]]
        return KNOWN_GATES[upper_case_name]
    
    n = ln2(len(initial_state))
    state = initial_state
#     print('initial state: ', state)

    for gate_operation in program:
        gate = gate_operation["gate"]
#         print('gate: ', gate)
        target = gate_operation["target"]
#         print('target: ', target)
        matrix_operator = get_operator(n, gate, target)
#         print('matrix: ', matrix_operator)
        
        state = np.dot(matrix_operator, state)
#         print('state: ', state)

        
#     print('final state: ', state)
    return state

In [22]:
class TestRunProgram(unittest.TestCase):

    def test_run_program_bell_00(self):
        state = np.array([1, 0, 0, 0])
        my_circuit = [
            { "gate": "h", "target": [0] },
            { "gate": "cx", "target": [0, 1] }
        ]
        np.testing.assert_allclose(
            list(run_program(state, my_circuit)),
            list(np.array([0.70710678, 0, 0, 0.70710678]))
            , rtol=1e-5, atol=0
        )

    def test_run_program_uniform_superposition(self):
        state = np.array([1, 0, 0, 0])
        my_circuit = [
            { "gate": "h", "target": [0,1] }
        ]
        np.testing.assert_allclose(
            list(run_program(state, my_circuit)),
            list([0.5]*4)
            , rtol=1e-5, atol=0
        )

suite = unittest.TestLoader().loadTestsFromTestCase(TestRunProgram)
unittest.TextTestRunner().run(suite)

..
----------------------------------------------------------------------
Ran 2 tests in 0.004s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

# Measure All [DONE; remove comments]

In [11]:
def measure_all(state_vector):
    # choose element from state_vector using weighted random and return it's index
    def get_probability(amplitude):
        pa = np.abs(amplitude)**2
        return pa
    
    probabilities = np.fromiter((get_probability(amplitude) for amplitude in state_vector), state_vector.dtype) 
#     print('prob', probabilities)
    
    result_array = multinomial(1, probabilities)
#     print('results', result_array)    
    indices = np.where(result_array == 1)
#     print('indices: ', indices)    
    index = indices[0][0]
#     print('index: ', index)
    
    bin_index = bin(index)[2:].zfill(ln2(len(state_vector)))
#     print('bin index: ', bin_index)
    
    return bin_index

In [12]:
class TestMeasureAll(unittest.TestCase):

    def test_measure_bell_00(self):
        actualResult = measure_all(np.array([0.70710678, 0, 0, -0.70710678]))
        self.assertTrue(
           (actualResult == '00') or (actualResult == '11')
        )

suite = unittest.TestLoader().loadTestsFromTestCase(TestMeasureAll)
unittest.TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

# Get Counts [remove comments + fix test]

In [13]:
def get_counts(state_vector, num_shots):
    # simply execute measure_all in a loop num_shots times and
    # return object with statistics in following form:
    #   {
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      ...
    #   }
    # (only for elements which occoured - returned from measure_all)

    counts = {} 
    # or listofzeros = [0] * len(state_vector)
    for x in range(num_shots):
        index = measure_all(state_vector)
        counts[index] = counts.get(index, 0) + 1
        # counts[measure_all(state_vector)] += 1 
        
    sorted_counts = json.dumps(counts, sort_keys=True, indent=4)
#     print(sorted_counts)

    # time considerarions of dic vs list
    # list needs extra pass to covert index to binary
    # dic can take binary string from measure all, but does this affect performance of get_counts
    # dic->json goves output in required format
    
    return sorted_counts

In [14]:
class TestGetCounts(unittest.TestCase):

    def test_run_1000_shots(self):
        self.assertEqual(
            get_counts(np.array([0.70710678, 0, 0, -0.70710678]), 1000),
            0
        )

suite = unittest.TestLoader().loadTestsFromTestCase(TestGetCounts)
unittest.TextTestRunner().run(suite)

F
FAIL: test_run_1000_shots (__main__.TestGetCounts)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-14-84dd1cb2a53b>", line 4, in test_run_1000_shots
    self.assertEqual(
AssertionError: '{\n    "00": 495,\n    "11": 505\n}' != 0

----------------------------------------------------------------------
Ran 1 test in 0.107s

FAILED (failures=1)


<unittest.runner.TextTestResult run=1 errors=0 failures=1>

# End-to-end example

In [15]:
# Define program:

my_circuit = [
{ "gate": "h", "target": [0] }, 
{ "gate": "cx", "target": [0, 1] }
]


# Create "quantum computer" with 2 qubits (this is actually just a vector :) )

my_qpu = get_ground_state(2)


# Run circuit

final_state = run_program(my_qpu, my_circuit)


# Read results

counts = get_counts(final_state, 1000)

print(counts)

# Should print something like:
# {
#   "00": 502,
#   "11": 498
# }

# Voila!

initial state:  [1. 0. 0. 0.]
Traceback (most recent call last):
  File "<ipython-input-15-c39b2474b0e4>", line 16, in <module>
    final_state = run_program(my_qpu, my_circuit)
  File "<ipython-input-9-c3e3323a22c2>", line 22, in run_program
    matrix_operator = get_operator(n, gate, target)
  File "<ipython-input-7-d55a1c308f18>", line 18, in get_operator
    is_a_controlled_gate = gate_unitary.upper()[0] == 'C'
AttributeError: 'numpy.ndarray' object has no attribute 'upper'

Use %tb to get the full traceback.


# Test execution

In [ ]:
unittest.main(argv=[''], verbosity=0, exit=False)